In [1]:
#### importing dependencies

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential, model_from_yaml # to save our weights and load the pre-trained weights model
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import sys
import io

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#### Importing the dataset

def read_data(path):
    print("Loading text data...")
    text = io.open(path, encoding='utf-8').read().lower()
    print("Done.")
    print("Corpus Length:", len(text))
    
    return text

In [3]:
text=read_data('threemenboat.txt')

Loading text data...
Done.
Corpus Length: 360663


In [4]:
#### Calculating the number of characters stored and building the c_to_i dictionary

chars=sorted(list(set(text)))

print('number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Tx = 40 # sequence length

number of unique characters in the corpus: 60


In [5]:
#### Helper functions to build (train examples & labels) the dataset & vectorize it

def build_data(text, Tx = 40, steps = 3):
    """Create a training set by scanning a window of size Tx over the text corpus, with a stride of 3."""
    X = [] # training examples (list)
    Y = [] # training labels (list)
    for i in range(0, len(text) - Tx, steps):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])
    print('number of training examples:', len(X))
    return X, Y


def vectorization(X, Y, n_x, char_indices, Tx = 40):
    """Convert X and Y (lists) into arrays to be given to a recurrent neural network."""
    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool) # array of shape (m, Tx, len(chars))
    y = np.zeros((m, n_x), dtype=np.bool) # array of shape (m, len(chars))
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
        
    return x, y 


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out
    

In [6]:
#### Building the data and Vectorizing it

X, Y = build_data(text, Tx=40, steps=3)
x, y = vectorization(X, Y, n_x = len(chars), char_indices = char_indices)


number of training examples: 120208


In [7]:
#### Defining the Model

print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(Tx, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
#### Function invoked at end of each epoch. Prints generated text.

def on_epoch_end(epoch, logs, maxlen=Tx):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    diversity=0.4

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    #usr_input = input("Write the beginning sentence and machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    #sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    #generated += usr_input
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(1000):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if next_char == '\n':
            continue

In [9]:
#### Generating the output

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=18,
          callbacks=[print_callback])

Epoch 1/18
120192/120208 [============================>.] - ETA: 0s - loss: 2.0815
----- Generating text after Epoch: 0
----- Generating with seed: "g out with me.

it seemed an excessive p"
g out with me.

120208/120208 [==============================] - 72s 603us/step - loss: 2.0815e had dot letter of any the had he started on the come to the had one so get and we had he had dight, and streen the have had it made of the had he had bet the harris of the have harris said.  i cet and the the have me the harris of the harris of and this me, and mate we had of the harr stree of and the had were seem the have and he had one and stear and were to her of it was and we had harris man in the harris the had he had bet me-lidetered of him and the were he had he had and you had harris and was becone with houle harris to all the were steary so the harrid in the have still out of the have he were and hand it sover the have he had it as the harris for the startion have the have harris done to the se

120208/120208 [==============================] - 78s 646us/step - loss: 1.4401y and in the boat of the whole and the things and not been to the only thing in the morning to the seem the setcher and bury part of the still the gentle one is a look and bed such and which has father in a lother and we had had and the things and the sail all the steer, and the rest a part of the man which the things.  and the steer and in the setching to give the state things to the sight.  it was all the pills.  it was a boat in the stirt of the bottom the way and she could think it was a long and the things and as we had don’t can sound and the stand that i had see expect at the way and all th
Epoch 8/18
120192/120208 [============================>.] - ETA: 0s - loss: 1.4181
----- Generating text after Epoch: 7
----- Generating with seed: "d twinkling watch-fires on
each height a"
d twinkling watch-fires on
each height and the man
and the pain of the man a bank of the work put the cheeses and stating at t

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log


 once the river and had with the sword of the fate of the boat, and the took a fair four better up on the billing to do a more and shad in a most me to be a lother of the water of a pow of a famour of the standing to the bath the river would come offfactial passes come off.

120208/120208 [==============================] - 89s 741us/step - loss: 1.3524and sand of a boat the great meat boat to me on the most me the discuse of the boat and shall, and she had a crick and said the boat and constitbods the senstance and sand down to the shad a champer, and said we were the setch for the boat and the seemed and be done of the boat and back and in the conse; but i shall do to be a moster for the river and said in the
Epoch 14/18
120192/120208 [============================>.] - ETA: 0s - loss: 1.3440
----- Generating text after Epoch: 13
----- Generating with seed: "bs them, and, perhaps, amidst the gall,
"
bs them, and, perhaps, amidst the gall,
when the silently than we were the told the mor

In [11]:
# Saving weights and model

model_yaml = model.to_yaml()
with open("threemen_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

model.save_weights("threemen_weights.h5")
print("Saved model to disk")


Saved model to disk
